# Decorators and Closures

The idea: **function decorators** let us "mark" functions in the source code to enhance their behaviour in some way. Now, mastering this requires understanding **closures** - which is what we get when functions capture variables defined outside of their bodies. (This reminds me also of lambdas in C++). 

If you want to implement your own function decorators, one needs to understand **closures** and then the need for `nonlocal` becomes obvious. 

The goal of this notebook is to explain:

- how function decorators work from the simplest registration to the rather more complicated parameterized ones. 

### Decorators 

*Def* A decorator is a **callable** that takes another function as an argument (the decorated function). 

The decorator might perform some processing with the decorated function, are returns it or replaces it with another function **or callable object.**

This code ...
```python
@decorate
def target():
    print('running target()')
```

... has the same effect as writing this:
```python
def target():
    print('running target()')

target = decorate(target)
```

In [ ]:
# example 
def deco(func):
    def inner():
        print("running inner()")
    return inner

@deco 
def target():
    print("running target()")
    
target() # equivalent to deco(target)()

running inner()


In [5]:
target # equivalent to deco(target)

<function __main__.deco.<locals>.inner()>

### When Python executes decorators

In [8]:
registry = [] # will hold references to functions decorated with @register

def register(func):
    print(f"running register({func})")
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')
    
def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

running register(<function f1 at 0x7a7e187b5000>)
running register(<function f2 at 0x7a7e187b4ee0>)


In [9]:
if __name__ == '__main__':
    main()

running main()
registry -> [<function f1 at 0x7a7e187b5000>, <function f2 at 0x7a7e187b4ee0>]
running f1()
running f2()
running f3()


What the above snippet of code shows is that a key feature of decorators is that the Python interpreter executes them right after the decorated function is defined. This is usually at *import time*, when a module is loaded by Python.

**Function decorators are executed as soon as the module is imported, but the decorated functions only run when they are explicitly invoked.**

### Variable Scope Rules

See chapter

### Closures

*Def* A **closure** is a function `f` with an extended scope that encompasses variables referenced in the body of `f` that are not global variables or local variables of `f`. Such variables must come from the local scope of an outer function that encompasses `f`. 

Consider an `avg` function to compute the mean of an ever-growing series of values, for example the average closing price of a stock over its entire history. 

In [10]:
class Averager():
    
    def __init__(self):
        self.series = []
        
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total / len(self.series)
    

The `Averager` class creates instances that are callable. THe history of previous values is stored as an attribute of the callable instance class.

In [11]:
avg = Averager()
avg(10) # 10.0
avg(11) # 10.5
avg(12) # 11.0

11.0

Now, the following example is a functional implementation, using the higher-order function `make_averager`.

In [12]:
def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)
    
    return averager

In [14]:
avg = make_averager()

print(avg(10)) # 10.0
print(avg(11)) # 10.5
print(avg(12)) # 11.0

10.0
10.5
11.0


Note the similarities. We call `Averager()` or `make_averager()` to get a callable object `avg` that will update the historical series and calculate the current mean. 

**Note**: It is obvious where the `avg` of the `Averager` class keeps its history: the `self.series` instance attribute. But where does the `avg` function in the second example find the `series`? 

As illustrated below the returned `averager` object shows how Python keeps the names of local and free variables in the `__code__` attribute that represents the compiled body of the function. 

In [15]:
print(avg.__code__.co_varnames) # ('new_value', 'total')
print(avg.__code__.co_freevars) # ('series',) is a free variable, not a local variable of averager() but it is used by averager() and defined in the enclosing scope of make_averager()

('new_value', 'total')
('series',)


The value for series is kept in the `__closure__` attribute of the returned function `avg`. Each item in `avg.__closure__` corresponds to
a name in `avg.__code__.co_freevars`. These items are cells, and they have an attribute called `cell_contents` where the actual value can
be found.

In [16]:
print(avg.__code__.co_freevars)
print(avg.__closure__) # (<cell at 0x7f8c9c1e5b80: list object at 0x7f8c9c1e5b50>,)
print(avg.__closure__[0].cell_contents) # [10, 11, 12

('series',)
(<cell at 0x7a7e18762f20: list object at 0x7a7e18603380>,)
[10, 11, 12]


To summarise: a **closure** is a function that retains the bindings of the free variables that exist when the function is defined, so that they can be used later when the function is invoked and the defining scope is no longer available.

### The nonlocal Declaration

Just look at example 9.12, 9.13 and it will all make sense

### Implementing a simpler decorator

In [23]:
# A decorator that clocks every invocation of the decorated function and displays the elapsed time, 
# the arguments passed, and the result of the call 
import time 

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print(f'[{elapsed:0.8f}s] {name}({arg_str}) -> {result!r}')
        return result # returns the result of the call to the decorated function
    return clocked

In [ ]:
@clock 
def snooze(seconds):
    time.sleep(seconds)
    
@clock 
def factorial(n):
    return 1 if n < 2 else n * factorial(n - 1)

# remember the above code actually does this def factoria ... factorial = clock(factorial)
print(factorial.__name__) # clocked, not factorial, because factorial is now clocked after the decoration


clocked


so factorial now holds a reference to the clocked function. From now on, each time factorial(n) is called, clocked(n) gets executed.

This is the typical behaviour of a decorator: it replaces the decorated function with a new function that accepts the same arguments and (usually) returns whatever the decorated function was supposed to return, while also doing some extra processing.

In [20]:

if __name__ == '__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123) # equivalent to clock(snooze)(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12327824s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000098s] factorial(1) -> 1
[0.00008103s] factorial(2) -> 2
[0.00009485s] factorial(3) -> 6
[0.00010487s] factorial(4) -> 24
[0.00011566s] factorial(5) -> 120
[0.00012730s] factorial(6) -> 720
6! = 720


### Powerful decorators in the standard library 

- `@cache`
- `@lru_cache`
- `@singledispatch`